In [ ]:
# Instalacja bibliotek
!pip install tensorflow matplotlib numpy

# Importowanie bibliotek
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from google.colab import files
import cv2
from PIL import Image
import io

# Wczytanie CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
train_images = train_images / 255.0
test_images = test_images / 255.0

# Wyświetlenie przykładów
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(train_images[i])
    plt.title(class_names[train_labels[i][0]])
    plt.axis('off')
plt.show()

# Budowa modelu CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

# Trenowanie modelu
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"\nDokładność na zbiorze testowym: {test_acc:.4f}")

# Wykresy dokładności i straty
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Dokładność (trening)')
plt.plot(history.history['val_accuracy'], label='Dokładność (walidacja)')
plt.title('Dokładność modelu')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Strata (trening)')
plt.plot(history.history['val_loss'], label='Strata (walidacja)')
plt.title('Strata modelu')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()
plt.show()

# Predykcje i macierz pomyłek
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(test_images[i])
    plt.title(f"Rzeczywiste: {class_names[test_labels[i][0]]}\nPredykcja: {class_names[predicted_labels[i]]}")
    plt.axis('off')
plt.show()
cm = confusion_matrix(test_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
plt.figure(figsize=(10, 10))
disp.plot(cmap='Blues')
plt.title('Macierz pomyłek')
plt.xticks(rotation=45)
plt.show()

# Klasyfikacja własnego obrazu
def classify_uploaded_image(model, class_names):
    uploaded = files.upload()
    for fn in uploaded.keys():
        img = Image.open(io.BytesIO(uploaded[fn]))
        img = img.resize((32, 32))
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        prediction = model.predict(img_array)
        predicted_class = class_names[np.argmax(prediction)]
        plt.figure(figsize=(5, 5))
        plt.imshow(img_array[0])
        plt.title(f"Predykcja: {predicted_class}")
        plt.axis('off')
        plt.show()

print("\nPrześlij obraz do klasyfikacji:")
classify_uploaded_image(model, class_names)

# Transfer learning z MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model_transfer = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')  # Poprawiona linia
])
model_transfer.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
history_transfer = model_transfer.fit(train_images, train_labels, epochs=5,
                                     validation_data=(test_images, test_labels))
test_loss_transfer, test_acc_transfer = model_transfer.evaluate(test_images, test_labels)
print(f"\nDokładność modelu transfer learning: {test_acc_transfer:.4f}")